# Title
[]()

In [9]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
import json
# from plotly.subplots import make_subplots
import requests
from datetime import time, datetime, timedelta
import pickle
import streamlit as st
import re

In [3]:
with open("credentials.json") as f:
    credentials = json.load(f)

# ig_user_id = credentials['am_ig_user_id']
# access_token = credentials['am_ig_access_token']
ig_user_id = credentials['mf_ig_user_id']
access_token = credentials['mf_access_token']

In [ ]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Account issues

In [13]:
# @st.cache_data
def get_ig_account_insights(ig_user_id, access_token, since=None, until=None, 
    filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\raw',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-02 16:13
    Get the daily impressions and reach a given Instagram account.

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    
    since_parameter = None
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    if since_parameter:
        url_without_token += f'&since={datetime.timestamp(since_parameter)}'
    else:
        url_without_token += f'&since={datetime.timestamp(since)}'
        since_parameter = since + timedelta(days=1)

    url = url_without_token+'&access_token='+access_token
    print(url_without_token)
    
    response_json_dict = dict()
    df_list = []
    earliest_end_time = None
    page = 1
    while (since_parameter > since):
        response = requests.get(url)
        print(f'Requesting page {page}...')
        print('\tResponse status code: ',response.status_code)
        response_json_dict[page] = response.json()
        if response.status_code//100 != 2: # Stop the function if there is an error in the request
            print(response_json_dict[page]['error'])
            break
        try:
            df_list.append(
                pd.concat([
                json_normalize(response_json_dict[page]['data'][0], record_path='values', record_prefix='impressions_'), # Impressions: "Total number of times the Business Account's media objects have been viewed"
                json_normalize(response_json_dict[page]['data'][1], record_path='values', record_prefix='reach_') # Reach: "Total number of times the Business Account's media objects have been uniquely viewed"
                ], axis=1)
            )
        except:
            print('No data in request response for page', page)
        earliest_end_time = response_json_dict[page]['data'][0]['values'][0]['end_time']
        since_parameter = datetime.strptime(re.sub(r'(.+)T.+', r'\1', earliest_end_time), "%Y-%m-%d")
        print('since_parameter: ',since_parameter)

        try:
            next_endpoint = response_json_dict[page]['paging']['previous']
            if next_endpoint+access_token != url:
                url = next_endpoint
            else:
                print('end')
                break
        except: 
            break
        page +=1
    try:
        df = pd.concat(df_list)
        df = df.reset_index(drop=True)
        print('Number of days of data:',len(df))
        new_response_json_dict = dict()
        for page, response in response_json_dict.items():
            new_response_json_dict[page] = {'data': response['data']}
    except:
        df = df_list 
    if filename:
        filename += '_account_insights'
        try:
            savepickle(df,filename+'_df','sav',csv_path)
            savepickle(new_response_json_dict,filename,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df, response_json_dict

# @st.cache_data
def update_ig_account_insights(ig_user_id, access_token, since=None, until=None,
    timestamp_column_suffix='end_time', filename=None,
    json_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\API_response',
    csv_path=r'C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\data\interim'):
    """ 
    2023-03-15 1:22
    Get the daily impressions and reach a given Instagram account. 
    Load any results that were previously saved (pull new data if no previously saved results available).

    Parameters:
        - ig_user_id: Can be obtained from Facebook Graph API explorer using this query 
            (requires business_management permission, possibly others also): 
             me/accounts?fields=instagram_business_account{id,name,username,profile_picture_url}
        - access_token
        - since and until (str): Date in 'yyyy-mm-dd format', e.g. '2023-01-01'. 
            Note: There cannot be more than 30 days (2592000 s) between since and until
        - timestamp_column_suffix (str): Suffix of the timestamp columns. default is 'end_time'. 
            Required to parse out the date range of the previously saved outputs.
        - filename (str): Filename (without extension) for saving the outputs. If None, outputs are not saved.
            For outputs to be saved, the custom functions save_csv and savepickle must be imported.
        - json_path and csv_path (raw string): path to which to save the json and dataframe outputs,
            respectively.
    
    Returns
        - df: DataFrame with the following information:
            - 
        - response_json: JSON object with each page number of results as the key (starting with 1)
    Example syntax:
    """
    previous_since, previous_until = None, None
    if filename:
        filename2 = f'{filename}_account_insights'
    try:
        df = loadpickle(filename2+'_df.sav', csv_path)
        df = df.reset_index(drop=True)
        timestamp_column = df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]
        df = df.sort_values(timestamp_column)
        response_json_dict = loadpickle(filename2+'.sav', json_path)
        previous_since = datetime.strptime(df.iloc[0][timestamp_column], "%Y-%m-%dT%H:%M:%S%z") # the %z format code is to indicate timezone as an offset
        previous_until = datetime.strptime(df.iloc[-1][timestamp_column], "%Y-%m-%dT%H:%M:%S%z")
        print('previous since date:', previous_since)
        print('previous until date:', previous_until)
    except:
        print('Unable to load prior results; making new API calls for entire date range.')
    
    url_root = "https://graph.facebook.com/v15.0/"
    url_without_token = f'{url_root}{ig_user_id}/insights?metric=impressions%2Creach&metric_type=time_series&period=day'
    
    if since:
        if type(since) == str:
            since = datetime.strptime(since, "%Y-%m-%d")
        else:
            default_time = time(0,0)
            since = datetime.combine(since, default_time)
    
    if until:
        if type(until) == str:
            until = datetime.strptime(until, "%Y-%m-%d")
        else:
            default_time=time(0,0)
            until = datetime.combine(until, default_time)
        if (until != datetime.now()) & (since != datetime.now()) & ((until - since).days > 30):
            since_parameter = until - timedelta(days=30)
        url_without_token += f'&until={datetime.timestamp(until)}'
    
    if (previous_since == None) & (previous_until == None):
        df, response_json_dict = get_ig_account_insights(
            ig_user_id, access_token, since=since, until=until, filename=filename,
            json_path=json_path)
        return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict
    elif previous_since == None:
        previous_since = since + timedelta (days=1)
        print('Previous `since` parameter could not be found; default to since + 1.')
    elif previous_until == None:
        previous_until = until - timedelta (days=1)
        print('Previous `until` parameter could not be found; default to until - 1.')
    if (previous_since.date() > since.date()):
        print(f'\nFetching older account insights from {datetime.strftime(since, "%Y-%m-%d")} to {datetime.strftime(previous_since, "%Y-%m-%d")}')
        older_insights_df, older_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token,
            json_path=json_path, since=since, until=previous_since)
        try:
            df = pd.concat([df.copy(), older_insights_df])
        except:
            df = older_insights_df
        try:
            # Update the keys of *response_json_dict* before merging with older_insights_response_json_dict. That way, final 
                # response dictionary always has insights from oldest dates first
            response_json_dict = dict( 
                zip([key+len(older_insights_response_json_dict) for key in response_json_dict.keys()], response_json_dict.values())
                )
            response_json_dict = {**older_insights_response_json_dict, **response_json_dict}
        except:
            response_json_dict = older_insights_response_json_dict
    if (previous_until.date() < until.date()):
        print(f'\nFetching newer account insights from {datetime.strftime(previous_until, "%Y-%m-%d")} to {datetime.strftime(until, "%Y-%m-%d")}')
        new_insights_df, new_insights_response_json_dict = get_ig_account_insights(ig_user_id, access_token, 
            json_path=json_path, since=previous_until, until=until)
        try:
            df = pd.concat([df.copy(), new_insights_df])
        except:
            df = new_insights_df
        new_insights_response_json_dict = dict( # Update the keys of new_insights_response_json_dict before merging with previous dict
            zip([key+len(response_json_dict) for key in new_insights_response_json_dict.keys()], new_insights_response_json_dict.values())
            )
        response_json_dict = {**response_json_dict, **new_insights_response_json_dict}
        # remove items containing API key as this will invalidate access tokens if published to Github
        new_response_json_dict = dict()
        for page, response in response_json_dict.items():
            new_response_json_dict[page] = {'data': response['data']}
    
    if (previous_until.date() >= until.date()) & (previous_since.date() <= since.date()):
        print('\nLoading previous saved results; no new API calls required.\n')
        new_response_json_dict = response_json_dict
        
    if filename:
        try:
            savepickle(df, filename2+'_df', 'sav', csv_path)
            savepickle(new_response_json_dict,filename2,'sav',json_path)
        except:
            print('Unable to save outputs')
    return df.sort_values(df.columns[df.columns.str.contains('_'+timestamp_column_suffix)][0]).reset_index(drop=True), response_json_dict

iteration = 1
data = dict()
response = dict
since = '2023-03-01'
until = '2023-04-01'
data[iteration], response[iteration] = get_ig_account_insights(
    ig_user_id, access_token,
    since=since, until=until
    )


https://graph.facebook.com/v15.0/17841400352707412/insights?metric=impressions%2Creach&metric_type=time_series&period=day&until=1680332400.0&since=1677744000.0
Requesting page 1...
	Response status code:  400
{'message': '(#10) Application does not have permission for this action', 'type': 'OAuthException', 'code': 10, 'fbtrace_id': 'AQkRuGBOW0IYE3YSU6M9ipp'}


TypeError: 'type' object does not support item assignment

# Plotting issues
Adjust `plot_account_insights` so there won't be an error message if there are days of the week without any posts.

In [ ]:
def plot_account_insights(
        input_df, agg='sum',
        metric_column_suffix='value', timezone='Canada/Pacific',
        posts_df=None,
        streamlit=False):
    
    """
    2023-03-02 16:07
    """
    df = process_account_insights(input_df, timezone=timezone)
    metrics = df.columns[df.columns.str.contains('_'+metric_column_suffix)].tolist()
    try:
        metrics += ['posts'] if len(posts_df)>0 else ''
    except:
        pass
    metrics = [metric.replace('_'+metric_column_suffix, '') for metric in metrics]
    df.columns = df.columns.str.replace('_'+metric_column_suffix, '')
    groupby_options = ['year-month', 'year-week', 'day_of_week', 'date']
    subplot_titles = [
        f'{metric} per {groupby} ({agg})' for groupby in groupby_options for metric in metrics]
    fig = make_subplots(
        rows=len(groupby_options)*len(metrics), cols=1, subplot_titles=subplot_titles
        )
    row = 1
    xtick_list = []
    df_list = []
    for groupby in (groupby_options):
        df_grouped = df.filter(items=metrics+[groupby]).groupby(
            groupby).agg(agg) if groupby !='date' else df.set_index('date')
        if groupby == 'day_of_week':
            day_names = df.sort_values('day_of_week')['day_of_week_name'].unique()
            df_grouped.index = [day[:3] for day in day_names]
            df_grouped = df_grouped/len(df['year-week'].unique())
        df_list.append(df_grouped)
        for metric in (metrics):
            if (metric == 'posts'):               
                posts_grouped = posts_df.filter(items=['caption']+[groupby]+['day_of_week_name']).groupby(
                    groupby).agg('count') 
                if groupby == 'date':
                    posts_grouped = posts_grouped.asfreq('D').fillna(0)
                elif groupby == 'day_of_week':
                    # posts_grouped.index = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
                    # print(posts_grouped.columns)
                    day_names2 = posts_grouped.sort_values('day_of_week')['day_of_week_name'].unique()
                    print('day_names', day_names2)
                    posts_grouped.index = [day[:3] for day in day_names]
                    posts_grouped = posts_grouped/(len(df['year-week'].unique()) if agg=='mean' else 1)
            fig.add_trace(
                go.Scatter(
                    y=df_grouped[metric] if metric !='posts' else posts_grouped['caption'],
                    x=df_grouped.index if metric !='posts' else posts_grouped.index, 
                    showlegend=False,
                ),
                row=row, col=1
            )
            row += 1
            if len(df_grouped.index) <= 24:
                xtick_list.append(df_grouped.index if metric != 'posts' else posts_grouped.index)
            else:
                xtick_list.append(None)

    fig.update_layout(
        title_text=f'Insights (periods end at {df.loc[0,"hour"]}:00 {timezone} time)',
        title_xanchor='center', title_x=0.5,
        height = len(groupby_options)*len(metrics) * 200,
        template='plotly'
        )
    # Update the xtick labels for each subplot
    for ax in fig['layout']:
        if ax.startswith('xaxis'):
            subplot = int(ax[5:]) if ax[5:] else 1
            fig['layout'][ax]['tickvals'] = xtick_list[subplot-1]
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return fig


## Final version

In [ ]:
def plot_account_insights(
        input_df, agg='sum',
        metric_column_suffix='value', timezone='Canada/Pacific',
        posts_df=None,
        streamlit=False):
    
    """
    2023-03-02 16:07
    """
    df = process_account_insights(input_df, timezone=timezone)
    metrics = df.columns[df.columns.str.contains('_'+metric_column_suffix)].tolist()
    try:
        metrics += ['posts'] if len(posts_df)>0 else ''
    except:
        pass
    metrics = [metric.replace('_'+metric_column_suffix, '') for metric in metrics]
    df.columns = df.columns.str.replace('_'+metric_column_suffix, '')
    groupby_options = ['year-month', 'year-week', 'day_of_week', 'date']
    subplot_titles = [
        f'{metric} per {groupby} ({agg})' for groupby in groupby_options for metric in metrics]
    fig = make_subplots(
        rows=len(groupby_options)*len(metrics), cols=1, subplot_titles=subplot_titles
        )
    row = 1
    xtick_list = []
    df_list = []
    for groupby in (groupby_options):
        df_grouped = df.filter(items=metrics+[groupby]).groupby(
            groupby).agg(agg) if groupby !='date' else df.set_index('date')
        if groupby == 'day_of_week':
            day_names = df.sort_values('day_of_week')['day_of_week_name'].unique()
            df_grouped.index = [day[:3] for day in day_names]
            df_grouped = df_grouped/len(df['year-week'].unique())
        df_list.append(df_grouped)
        for metric in (metrics):
            if (metric == 'posts'):
                print('unique days of the week:', posts_df['day_of_week_name'].unique())               
                posts_grouped = posts_df.filter(items=['caption']+[groupby]).groupby(
                    groupby).agg('count') 
                if groupby == 'date':
                    posts_grouped = posts_grouped.asfreq('D').fillna(0)
                elif groupby == 'day_of_week':
                    day_map = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
                    posts_grouped = posts_grouped.rename(index=day_map)
                    posts_grouped = posts_grouped/(len(df['year-week'].unique()) if agg=='mean' else 1)
            fig.add_trace(
                go.Scatter(
                    y=df_grouped[metric] if metric !='posts' else posts_grouped['caption'],
                    x=df_grouped.index if metric !='posts' else posts_grouped.index, 
                    showlegend=False,
                ),
                row=row, col=1
            )
            row += 1
            if len(df_grouped.index) <= 24:
                xtick_list.append(df_grouped.index if metric != 'posts' else posts_grouped.index)
            else:
                xtick_list.append(None)

    fig.update_layout(
        title_text=f'Insights (periods end at {df.loc[0,"hour"]}:00 {timezone} time)',
        title_xanchor='center', title_x=0.5,
        height = len(groupby_options)*len(metrics) * 200,
        template='plotly'
        )
    # Update the xtick labels for each subplot
    for ax in fig['layout']:
        if ax.startswith('xaxis'):
            subplot = int(ax[5:]) if ax[5:] else 1
            fig['layout'][ax]['tickvals'] = xtick_list[subplot-1]
    if streamlit:
        st.plotly_chart(fig, use_container_width=True)
    else:
        fig.show()
    return fig

# *End of Page*